In [1]:
!wget 'https://raw.githubusercontent.com/Apress/applied-natural-language-processing-w-python/master/data_etc/deu.txt'

--2024-04-11 17:41:11--  https://raw.githubusercontent.com/Apress/applied-natural-language-processing-w-python/master/data_etc/deu.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12316220 (12M) [text/plain]
Saving to: 'deu.txt'

deu.txt             100%[===================>]  11.75M  --.-KB/s    in 0.1s    

2024-04-11 17:41:11 (116 MB/s) - 'deu.txt' saved [12316220/12316220]



In [2]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 76.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [3]:
import torch

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
print(device)

cuda


In [6]:
import torchdata.datapipes as dp
import torchtext.transforms as T
import spacy
import random
from torchtext.vocab import build_vocab_from_iterator
eng = spacy.load("en_core_web_sm") # Load the English model to tokenize English text
de = spacy.load("de_core_news_sm") # Load the German model to tokenize German text

In [7]:
import random

def split_dataset(input_file, train_file, test_file, train_percent):
    with open(input_file, 'r') as file:
        lines = file.readlines()  

    total_lines = len(lines)
    train_lines = int(total_lines * train_percent)
    test_lines = total_lines - train_lines


    random.shuffle(lines)

    with open(train_file, 'w') as train:
        for line in lines[:train_lines]:
            train.write(line)

    with open(test_file, 'w') as test:
        for line in lines[train_lines:]:
            test.write(line)


input_file = '/kaggle/working/deu.txt'


train_file = 'shiny_train.txt'
test_file = 'shiny_test.txt'

train_percent = 0.8

split_dataset(input_file, train_file, test_file, train_percent)

In [8]:
FILE_PATH = '/kaggle/working/shiny_train.txt'
data_pipe = dp.iter.IterableWrapper([FILE_PATH])
data_pipe = dp.iter.FileOpener(data_pipe, mode='rb')
data_pipe = data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)

In [9]:
for sample in data_pipe:
    print(sample)
    break

('Tom spent three weeks in the hospital.', 'Tom war drei Wochen im Krankenhaus.')


In [10]:
def engTokenize(text):
    """
    Tokenize an English text and return a list of tokens
    """
    return [token.text for token in eng.tokenizer(text)]

def deTokenize(text):
    """
    Tokenize a German text and return a list of tokens
    """
    return [token.text for token in de.tokenizer(text)]

In [11]:
print(engTokenize("Have a good day!!!"))
print(deTokenize("Haben Sie einen guten Tag!!!"))

['Have', 'a', 'good', 'day', '!', '!', '!']
['Haben', 'Sie', 'einen', 'guten', 'Tag', '!', '!', '!']


In [12]:
def getTokens(data_iter, place):
    """
    Function to yield tokens from an iterator. Since, our iterator contains
    tuple of sentences (source and target), `place` parameters defines for which
    index to return the tokens for. `place=0` for source and `place=1` for target
    """
    for english, german in data_iter:
        if place == 0:
            yield engTokenize(english)
        else:
            yield deTokenize(german)

In [13]:
source_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,0),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
source_vocab.set_default_index(source_vocab['<unk>'])

In [14]:
target_vocab = build_vocab_from_iterator(
    getTokens(data_pipe,1),
    min_freq=2,
    specials= ['<pad>', '<sos>', '<eos>', '<unk>'],
    special_first=True
)
target_vocab.set_default_index(target_vocab['<unk>'])
    

In [15]:
print(source_vocab.get_itos()[:9])

['<pad>', '<sos>', '<eos>', '<unk>', '.', 'I', 'Tom', 'to', 'you']


In [16]:
def getTransform(vocab):
    """
    Create transforms based on given vocabulary. The returned transform is applied to sequence
    of tokens.
    """
    text_tranform = T.Sequential(
        ## converts the sentences to indices based on given vocabulary
        T.VocabTransform(vocab=vocab),
        ## Add <sos> at beginning of each sentence. 1 because the index for <sos> in vocabulary is
        # 1 as seen in previous section
        T.AddToken(1, begin=True),
        ## Add <eos> at beginning of each sentence. 2 because the index for <eos> in vocabulary is
        # 2 as seen in previous section
        T.AddToken(2, begin=False)
    )
    return text_tranform

In [17]:
temp_list = list(data_pipe)
some_sentence = temp_list[798][0]
print("Some sentence= ", end="")
print(some_sentence)
transformed_sentence = getTransform(source_vocab)(engTokenize(some_sentence))
print("Transformed sentence=", end="")
print(transformed_sentence)
index_to_string = source_vocab.get_itos()
for index in transformed_sentence:
    print(index_to_string[index], end=" ")

Some sentence= Why couldn't you do that?
Transformed sentence=[1, 94, 91, 12, 8, 15, 18, 10, 2]
<sos> Why could n't you do that ? <eos> 

In [18]:
def applyTransform(sequence_pair):
    """
    Apply transforms to sequence of tokens in a sequence pair
    """

    return (
        getTransform(source_vocab)(engTokenize(sequence_pair[0])),
        getTransform(target_vocab)(deTokenize(sequence_pair[1]))
    )
data_pipe = data_pipe.map(applyTransform) ## Apply the function to each element in the iterator
temp_list = list(data_pipe)
print(temp_list[0])

([1, 6, 638, 150, 969, 16, 9, 648, 4, 2], [1, 6, 35, 149, 1003, 69, 673, 4, 2])


In [19]:
def sortBucket(bucket):
    """
    Function to sort a given bucket. Here, we want to sort based on the length of
    source and target sequence.
    """
    return sorted(bucket, key=lambda x: (len(x[0]), len(x[1])))

In [20]:
bucket = [
    (['how', 'are', 'you'], ['wie', 'geht', 'es', 'dir']),
    (['hello', 'world'], ['hallo', 'welt']),
    (['this', 'is', 'a', 'test'], ['das', 'ist', 'ein', 'test'])
]

sorted_data = sortBucket(bucket)

for item in sorted_data:
    print(item)

(['hello', 'world'], ['hallo', 'welt'])
(['how', 'are', 'you'], ['wie', 'geht', 'es', 'dir'])
(['this', 'is', 'a', 'test'], ['das', 'ist', 'ein', 'test'])


In [21]:
data_pipe = data_pipe.bucketbatch(
    batch_size = 64, batch_num=5,  bucket_num=1,
    use_in_batch_shuffle=False, sort_key=sortBucket
)

In [22]:
print(list(data_pipe)[0])

[([1, 5, 451, 577, 6, 274, 905, 348, 195, 41, 4, 2], [1, 1746, 6, 292, 115, 4784, 505, 8, 2]), ([1, 73, 235, 52, 89, 10004, 25, 9, 64, 320, 4, 2], [1, 28, 244, 10, 7435, 3, 399, 4, 2]), ([1, 1850, 146, 215, 7, 34, 11, 911, 194, 184, 4, 2], [1, 3998, 312, 84, 1537, 289, 313, 4, 2]), ([1, 27, 297, 65, 12, 22, 102, 346, 11, 268, 4, 2], [1, 4629, 44, 22, 355, 9, 134, 50, 4, 2]), ([1, 6, 13, 11, 159, 519, 975, 105, 8, 37, 4, 2], [1, 6, 10, 21, 3826, 3368, 59, 13, 4, 2]), ([1, 5, 68, 108, 11, 938, 721, 56, 18, 842, 4, 2], [1, 66, 1485, 10, 26, 9, 251, 3, 4, 2]), ([1, 26, 19, 9, 371, 801, 2101, 17, 38, 132, 4, 2], [1, 23, 35, 22, 9432, 2438, 224, 100, 4, 2]), ([1, 27, 808, 13, 2382, 5539, 7, 38, 184, 121, 4, 2], [1, 66, 1797, 1507, 68, 41, 221, 857, 170, 4, 2]), ([1, 190, 310, 294, 98, 1587, 23, 44, 23, 13, 4, 2], [1, 261, 3396, 287, 318, 16, 521, 1207, 63, 4, 2]), ([1, 1877, 117, 36, 20, 324, 7, 742, 36, 64, 4, 2], [1, 2034, 5, 45, 119, 41, 1208, 11, 169, 4, 2]), ([1, 163, 8, 179, 4171, 16, 

In [23]:
def separateSourceTarget(sequence_pairs):
    """
    input of form: `[(X_1,y_1), (X_2,y_2), (X_3,y_3), (X_4,y_4)]`
    output of form: `((X_1,X_2,X_3,X_4), (y_1,y_2,y_3,y_4))`
    """
    sources,targets = zip(*sequence_pairs)
    return sources,targets

## Apply the function to each element in the iterator
data_pipe = data_pipe.map(separateSourceTarget)
print(list(data_pipe)[0])

(([1, 6, 1650, 4, 2], [1, 6, 19, 362, 4, 2], [1, 80, 84, 1092, 4, 2], [1, 73, 5332, 21, 4, 2], [1, 741, 14, 191, 4, 2], [1, 2501, 568, 21, 4, 2], [1, 5, 44, 989, 4, 2], [1, 6, 67, 499, 4, 2], [1, 5, 562, 747, 4, 2], [1, 5, 44, 2342, 4, 2], [1, 24, 53, 2996, 4, 2], [1, 3404, 9, 5060, 4, 2], [1, 5, 703, 70, 4, 2], [1, 24, 53, 1593, 4, 2], [1, 2052, 7, 29, 158, 2], [1, 24, 932, 6, 4, 2], [1, 6, 370, 2683, 4, 2], [1, 1783, 186, 82, 4, 2], [1, 1144, 5, 2314, 8, 10, 2], [1, 204, 13, 971, 90, 10, 2], [1, 73, 14, 39, 3134, 4, 2], [1, 49, 71, 8, 584, 10, 2], [1, 42, 14, 39, 128, 4, 2], [1, 888, 9, 1132, 10169, 10, 2], [1, 50, 52, 1005, 2093, 4, 2], [1, 5, 283, 30, 300, 4, 2], [1, 119, 8, 129, 23, 10, 2], [1, 3401, 9, 1534, 2558, 4, 2], [1, 5, 68, 89, 285, 4, 2], [1, 6, 585, 150, 893, 4, 2], [1, 50, 207, 39, 441, 4, 2], [1, 73, 19, 126, 2976, 4, 2], [1, 6, 13, 12, 801, 4, 2], [1, 27, 4507, 37, 1092, 4, 2], [1, 80, 84, 4060, 621, 4, 2], [1, 26, 13, 553, 1410, 4, 2], [1, 204, 91, 474, 23, 10, 2], 

In [24]:
def applyPadding(pair_of_sequences):
    """
    Convert sequences to tensors and apply padding
    """
    return (T.ToTensor(0)(list(pair_of_sequences[0])), T.ToTensor(0)(list(pair_of_sequences[1])))
## `T.ToTensor(0)` returns a transform that converts the sequence to `torch.tensor` and also applies
# padding. Here, `0` is passed to the constructor to specify the index of the `<pad>` token in the
# vocabulary.
data_pipe = data_pipe.map(applyPadding)

In [25]:
print(list(data_pipe)[0])

(tensor([[   1,  667, 2856,    4,    2,    0,    0,    0],
        [   1,   24,  256,    4,    2,    0,    0,    0],
        [   1,  436,   14, 1030,    4,    2,    0,    0],
        [   1,   50, 1899,   96,    4,    2,    0,    0],
        [   1,   40,   84, 3675,    4,    2,    0,    0],
        [   1,   40,   53,  292,    4,    2,    0,    0],
        [   1,    5,  108,  449,    4,    2,    0,    0],
        [   1,    5,   44, 1453,    4,    2,    0,    0],
        [   1,    6,   13, 5225,    4,    2,    0,    0],
        [   1,    6,   13, 3907,    4,    2,    0,    0],
        [   1,  116,  867,  489,   10,    2,    0,    0],
        [   1,    6,   13, 1649,    4,    2,    0,    0],
        [   1,  119,   23,  133,   10,    2,    0,    0],
        [   1,   80,  178,   21,    4,    2,    0,    0],
        [   1,  767,   13,  641,    4,    2,    0,    0],
        [   1,    6,   13,  596,    4,    2,    0,    0],
        [   1,    6,   13,  397,    4,    2,    0,    0],
        [   1

In [26]:
source_index_to_string = source_vocab.get_itos()
target_index_to_string = target_vocab.get_itos()

def showSomeTransformedSentences(data_pipe):
    """
    Function to show how the sentences look like after applying all transforms.
    Here we try to print actual words instead of corresponding index
    """
    for sources,targets in data_pipe:
        if sources[0][-1] != 0:
            continue # Just to visualize padding of shorter sentences
        for i in range(4):
            source = ""
            for token in sources[i]:
                source += " " + source_index_to_string[token]
            target = ""
            for token in targets[i]:
                target += " " + target_index_to_string[token]
            print(f"Source: {source}")
            print(f"Traget: {target}")
        break

showSomeTransformedSentences(data_pipe)

Source:  <sos> Tom asked about Mary . <eos> <pad> <pad>
Traget:  <sos> Tom hat sich nach Maria erkundigt . <eos> <pad> <pad> <pad>
Source:  <sos> Tom is getting bored . <eos> <pad> <pad>
Traget:  <sos> Tom fängt an , sich zu langweilen . <eos> <pad> <pad>
Source:  <sos> Everything changes over time . <eos> <pad> <pad>
Traget:  <sos> Alles ändert sich im Laufe der Zeit . <eos> <pad> <pad>
Source:  <sos> Your screams frightened me . <eos> <pad> <pad>
Traget:  <sos> Deine Schreie <unk> mich in Angst und Schrecken . <eos> <pad>


In [27]:
for (data, target) in data_pipe:
    print(data)
    print(data.shape)
    print(target)
    print(target.shape)
    break

tensor([[   1, 4271,  621,  ...,    0,    0,    0],
        [   1,   42,   85,  ...,    0,    0,    0],
        [   1,   49,   15,  ...,    0,    0,    0],
        ...,
        [   1,   27,  238,  ...,    4,    2,    0],
        [   1,    6,  108,  ...,    4,    2,    0],
        [   1,    6, 2458,  ..., 1735,    4,    2]])
torch.Size([64, 21])
tensor([[   1, 6309,  731,  ...,    0,    0,    0],
        [   1,   34,  172,  ...,    0,    0,    0],
        [   1,   62,  234,  ...,    0,    0,    0],
        ...,
        [   1,  377, 3271,  ...,    0,    0,    0],
        [   1,    6, 5115,  ...,    0,    0,    0],
        [   1,    6, 1675,  ...,    0,    0,    0]])
torch.Size([64, 22])


In [28]:
import torch.nn as nn
import random
import torch.optim as optim
from tqdm import tqdm
import torch

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [30]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, p):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers,bidirectional=True)
        self.fc_hidden = nn.Linear(hidden_size*2, hidden_size)
        self.fc_cell = nn.Linear(hidden_size*2, hidden_size)
        
    def forward (self, x):
        #  sequence_lenght x batch_size 
        embedding = self.dropout(self.embedding(x))
        # embedding shape => (sequence_lenght,batch_size,embedding_size)
        encoder_states , (hidden, cell) = self.rnn(embedding)
        # hidden shape = (2, Batch_size, hidden_size)
        hidden = self.fc_hidden(torch.cat((hidden[0:1], hidden[1:2]), dim=2))
        # hidden shape = (Batch_size, 2*hidden_size)
        cell = self.fc_cell(torch.cat((cell[0:1], cell[1:2]), dim=2))

        return encoder_states, hidden, cell
    
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, output_size,
                num_layers, p):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(p)
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.LSTM(hidden_size*2 + embedding_size, hidden_size, num_layers)
        self.energy = nn.Linear(hidden_size*3, 1)
        self.softmax = nn.Softmax(dim=0)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x,encoder_states, hidden, cell):
        #shape of x: (batch_size) but we want (1, batch_size)
        x = x.unsqueeze(0)
        embedding = self.dropout(self.embedding(x))
        # embedding shape: (1,batch_size, embedding_size)
        
        sequence_length = encoder_states.shape[0]
        
        h_reshape = hidden.repeat(sequence_length, 1, 1)
        energy = self.relu(self.energy(torch.cat((h_reshape, encoder_states), dim=2)))
        attention = self.softmax(energy)
        # attention shape = (sequence_lenght, Batch_size, 1)
        attention = attention.permute(1, 2, 0) # (N, 1, sequence_len)
        # Encoder states ()
        encoder_states = encoder_states.permute(1, 0, 2) # (N, seq_len, hidden_size*2)
        
        #torch.bmm
        # (N, 1, hidden_size*2) -> permute -> (1, N, hidden_size*2)
        # Performing the attention muplipy element wise
        context_vector = torch.bmm(attention, encoder_states).permute(1, 0, 2)
        
        rnn_input = torch.cat((context_vector, embedding), dim=2)
        outputs, (hidden, cell) = self.rnn(rnn_input, (hidden, cell))
        
        # outputs shape (1, N, hidden_size)
        predictions = self.fc(outputs)
        # predicitons shape: (1,batch_size, lenght_of_vocab)
        predictions = predictions.squeeze(0)
        
        return predictions, hidden, cell
    
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self, source, target, teacher_force_ratio=0.5):
        batch_size = source.shape[1]
        target_len = target.shape[0]
        target_vocab_size = len(target_vocab)
        
        outputs = torch.zeros(target_len, batch_size, target_vocab_size).to(device)
        
        encoder_states, hidden, cell = self.encoder(source)
        # Grab start token
        x = target[0]
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder(x,encoder_states, hidden, cell)
            # output shape: (batch_size, target_vocab_size)
            outputs[t] = output
            best_guess = output.argmax(1)
            
            x = target[t] if random.random() < teacher_force_ratio else best_guess
        return outputs

In [31]:
# Training hyper parameters
num_epochs = 20
learning_rate = 0.001
batch_size = 64
load_model = False
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
input_size_encoder = len(source_vocab)
output_size_decoder = len(target_vocab)
encoder_embedding_size = 300
decoder_embedding_size = 300
hidden_size = 1024
num_layers = 1
enc_dropout = 0.5
dec_dropout = 0.5

In [32]:
encoder_net = Encoder(input_size_encoder, encoder_embedding_size, hidden_size,num_layers,
                    enc_dropout).to(device)
decoder_net = Decoder(output_size_decoder, decoder_embedding_size, hidden_size,output_size_decoder, num_layers,
                     dec_dropout).to(device)

In [33]:
model = Seq2Seq(encoder_net, decoder_net).to(device)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = target_vocab.lookup_indices(['<pad>'])[0]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)

In [34]:
FILE_PATH = '/kaggle/working/shiny_test.txt'
test_data_pipe = dp.iter.IterableWrapper([FILE_PATH])
test_data_pipe = dp.iter.FileOpener(test_data_pipe, mode='rb')
test_data_pipe = test_data_pipe.parse_csv(skip_lines=1, delimiter='\t', as_tuple=True)

In [35]:
import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, german_voc, english_voc, spacy_en, device, max_length=50):
    
    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_en(sentence)]
    else:
        tokens = [token.lower() for token in sentence]
      
    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, '<sos>')
    tokens.append('<eos>')
    # Go through each german token and convert to an index
    text_to_indices = english_voc.lookup_indices(tokens)

    
    # Convert to Tensor
    sentence_tensor = torch.tensor(text_to_indices, device=device).unsqueeze(1)
    
    # Build encoder hidden, cell state
    
    with torch.no_grad():
        encoder_states, hidden, cell = model.encoder(sentence_tensor)
 
    outputs = german_voc.forward(['<sos>'])
    end_of_sentece = german_voc.forward(['<eos>'])[0]
    
    for _ in range(max_length):
        previous_word = torch.tensor([outputs[-1]], device=device)
        with torch.no_grad():
            output, hidden, cell = model.decoder(previous_word, encoder_states, hidden, cell)
            best_guess = output.argmax(1).item()
        outputs.append(best_guess)
    
    
        # Model predicts it's the end of the sentence
        if best_guess == end_of_sentece:
            break
    
    translated_sentence = german_voc.lookup_tokens(outputs)
    # remove start token
    return translated_sentence[1:]

In [36]:
sentence = 'You must know yourself.'
spacy_en = spacy.load("en_core_web_sm")

for epoch in range(num_epochs):
    
    model.eval()
    translated_sentence = translate_sentence(model, sentence, target_vocab, source_vocab,spacy_en, device)
    print(' '.join(translated_sentence[:-1]))
    model.train()
    print(f'Epoch {epoch+1} / {num_epochs}')
    for batch_idx, (data, target) in enumerate(data_pipe):
            data = data.permute([1,0])
            data = data.to(device)
            target = target.permute([1,0])
            target = target.to(device)
            output = model(data, target)
            # output shape: (target_len, batch_size, output_len)
            output = output[1:].reshape(-1, output.shape[2])
            target = target[1:].reshape(-1)
            
            optimizer.zero_grad()
            loss = criterion(output, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
            optimizer.step()     

baute entlassen lagen Forelle Spezialität 1969 durchs stirbst Landhaus schlägst hinkommt hinkommt hinkommt Amtes Wecker Begehe Ziegen trösteten Sumo Kern Vorort eingesteckt zahm Schreck viele Komitees Kochbuch Kern gekommen wirfst prüfen Schlag einfältig entsprechend Fahrenheit Notizbücher Töchter Anschuldigungen Gäste witzig fantasievoll 10.000 Bankier Unglückstag abschließt verlängerte erhängt Poet Korinthenkacker
Epoch 1 / 20
Niemand möchte wissen .
Epoch 2 / 20
Vielleicht müssen es selbst selbst .
Epoch 3 / 20
Es muss du selbst wissen .
Epoch 4 / 20
Jemand müssen es selbst wissen .
Epoch 5 / 20
Es muss dich selbst wissen .
Epoch 6 / 20
Jetzt müssen du selbst wissen .
Epoch 7 / 20
Nichts müssen es selbst kennen .
Epoch 8 / 20
Mehr muss du selbst kennen .
Epoch 9 / 20
Man muss dich selbst kennen .
Epoch 10 / 20
Man muss dich selbst kennen .
Epoch 11 / 20
Du müsst dich selbst kennen .
Epoch 12 / 20
Ansonsten müssen du selbst wissen .
Epoch 13 / 20
Damit müsst du selbst kennen .
Epoch 

In [37]:
from torchmetrics.functional.text import bleu_score
from torch.utils.data import DataLoader


def bleu(data_pipe, model, german_vocab, english_vocab, device):
    spacy_en = spacy.load("en_core_web_sm")
    targets = []
    outputs = []
    
    for idx, (source, target) in enumerate(data_pipe):
        if idx % 10000 == 0:
            print(idx)
        
        prediction = translate_sentence(model, source, german_vocab, english_vocab, spacy_en, device)
        prediction = prediction[:-1]
        detokenized  = ' '.join(deTokenize(target))
        targets.append([detokenized])
        outputs.append(" ".join(prediction))
#         print(f'Source: {source} Target: {t} Pred: {detokenized}', end='\n\n')
    return bleu_score(outputs, targets)

In [38]:
score = bleu(test_data_pipe, model, target_vocab, source_vocab, device)

0
10000
20000
30000


In [39]:
print(score)

tensor(0.1147)
